# Advanced XGBoost Hyperparameter Tuning

Gradient boosting machines (GBMs), and more specifically the XGBoost variant, is an extremely popular machine learning algorithm that have proven successful across many domains and is one of the leading algorithmic methods for tabular data. Whereas random forests build an ensemble of deep independent trees, GBMs build an ensemble of shallow trees in sequence with each tree learning and improving on the previous one. Although shallow trees by themselves are rather weak predictive models, they can be “boosted” to produce a powerful “committee” that, ___when appropriately tuned___, is often hard to beat with other algorithms.

The key phrase above to note is -- "_when appropriately tuned_." XGBoost has many hyperparameters to refine, which can be overwhelming. Moreover, refining these hyperparameters can take significant time; and the tracking of model performance based on hyperparameter values can get unwieldy if done inefficiently. Consequently, the objective of this tutorial is to illustrate:
<br><br>
1. Best practices for tuning XGBoost hyperparameters
2. Leveraging Hyperopt for an effective and efficient XGBoost grid search
3. Using MLflow for tracking and organizing grid search performance

## Assumptions

This tutorial makes the assumption that the reader:
<br><br>
1. Is comfortable using Python for basic data wrangling processes
2. Is comfortable writing Python functions and understands context managers
3. Understands the basics of the GBM and XGBoost algorithm
4. Has access to a Databricks ML runtime cluster

This tutorial also introduces the use of Hyperopt and MLflow for parts of the hyperparameter tuning process; however, this tutorial does not provide a comprehensive introduction to these packages. When appropriate this tutorial will hyperlink to additional resources that provide more comprehensive documentation.

## Prerequisites

This tutorial leverages the Databricks Community Edition with an 8.3 ML runtime cluster. 

### Packages

The following packages are used in this tutorial with the emphasis on [xgboost](https://xgboost.readthedocs.io/en/latest/), [hyperopt](http://hyperopt.github.io/hyperopt/), and [mlflow](https://mlflow.org/), all of which are pre-installed on the 8.3 ML cluster.

In [0]:
# helper packages
import pandas as pd
import numpy as np
import time
import warnings

# modeling
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import xgboost as xgb

# hyperparameter tuning
from hyperopt import fmin, tpe, hp, SparkTrials, STATUS_OK
from hyperopt.pyll import scope

# model/grid search tracking
import mlflow

In [0]:
# typically not advised but doing this to minimize excessive messaging 
# during the grid search
warnings.filterwarnings("ignore") 

### Data

For simplicity we will use the well known [wine quality dataset](https://archive.ics.uci.edu/ml/datasets/wine+quality) provided by the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php). Our objective in this modeling task is to use wine characteristics in order to predict the ___quality___ of the wine. In this example we will convert this to a classification problem with the objective of predicting if a wine is high quality ( \\(quality \\geq 7\\) ) or low quality ( \\(quality < 7\\) ). In addition, we will do a few other simple data wrangling tasks. Consquently, our data prep tasks include:
<br><br>
1. Read in the data
2. Create indicator column for red vs. white wine
3. Combine the red and white wine datasets
4. Clean up column names
5. Convert wine quality column to a binary response variable


In [0]:
# 1. read in data
white_wine = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv", sep=";")
red_wine = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv", sep=";")

# 2. create indicator column for red vs. white wine
red_wine['is_red'] = 1
white_wine['is_red'] = 0

# 3. combine the red and white wine data sets
data = pd.concat([red_wine, white_wine], axis=0)
 
# 4. remove spaces from column names
data.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)

# 5. convert "quality" column to 0 vs. 1 to make this a classification problem
data["quality"] = (data["quality"] >= 7).astype(int)

In [0]:
data.head(10)

Out[5]:

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,is_red
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,0,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0,1
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,0,1
6,7.9,0.60,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4,0,1
7,7.3,0.65,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,1,1
8,7.8,0.58,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5,1,1
9,7.5,0.50,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.80,10.5,0,1


Next, we'll split our data into train and test sets using the default 75% (train) 25% (test) ratio.

In [0]:
# split data into train (75%) and test (25%) sets
train, test = train_test_split(data, random_state=123)
X_train = train.drop(columns="quality")
X_test = test.drop(columns="quality")
y_train = train["quality"]
y_test = test["quality"]

In [0]:
X_train.head()

Out[7]:

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,is_red
1773,7.8,0.390,0.26,9.9,0.059,33.0,181.0,0.99550,3.04,0.42,10.9,0
1094,6.6,0.725,0.09,5.5,0.117,9.0,17.0,0.99655,3.35,0.49,10.8,1
4813,5.8,0.360,0.50,1.0,0.127,63.0,178.0,0.99212,3.10,0.45,9.7,0
853,9.3,0.360,0.39,1.5,0.080,41.0,55.0,0.99652,3.47,0.73,10.9,1
2185,7.5,0.180,0.31,11.7,0.051,24.0,94.0,0.99700,3.19,0.44,9.5,0


In [0]:
y_test.head()

Out[8]: 1321 0
1168 0
3470 1
4181 0
547 0
Name: quality, dtype: int64

And finally we will convert our data to a DMatrix object which is an XGBoost internal data structure that is optimized for both memory efficiency and training speed.

In [0]:
train = xgb.DMatrix(data=X_train, label=y_train)
test = xgb.DMatrix(data=X_test, label=y_test)

## XGBoost Grid Search

As previously mentioned XGBoost has many hyperparameters. These hyperparameters can be grouped into 4 categories. The following provides a quick overview of the primary hyperparameters along with general guidelines regarding the range of values to typically search across.

__Boosting hyperparameters__ include those parameters that are central to the gradient boosting algorithm. This primarily focuses on the [gradient descent process](https://bradleyboehmke.github.io/HOML/gbm.html#gbm-gradient). The two main parameters include learning rate (discussed below) and the number of trees. However, the number of trees is, technically, not a hyperparameter. We primarily want to ensure we are training for the right number of trees to minimize the loss function while not overfitting. XGBoost allows for some early stopping procedures (discussed later) that allows us to train for a large number of trees but then stop training once we no longer improve on the loss. Consequently, the primary boosting hyperparamter of concern is learning rate.

* __learning_rate__: Determines the contribution of each tree on the final outcome and controls how quickly the algorithm learns. Values range from 0–1 with typical values between 0.001–0.3. Smaller values make the model robust to the specific characteristics of each individual tree, thus allowing it to generalize well. Smaller values also make it easier to stop prior to overfitting; however, they increase the risk of not reaching the optimum with a fixed number of trees and are more computationally demanding. __Recommendation__: Search across values ranging from 0.0001-1 on a log scale (i.e. 0.0001, 0.001, 0.01, 0.1, 1).

__Tree hyperparameters__ - although we can perform boosting on non-tree based algorithms, using decision trees as the base learner for boosting is the most common procedure. Consequently, there are several hyperparamters that steer how the decision trees are built. The most common tree hyperparameters to tune include:

* __max_depth__: Controls the depth of the individual trees. Typical values range from a depth of 3–8 but it is not uncommon to see a tree depth of 1 ([J. Friedman, Hastie, and Tibshirani 2001](https://web.stanford.edu/~hastie/ElemStatLearn/)). Smaller depth trees such as decision stumps are computationally efficient (but require more trees); however, higher depth trees allow the algorithm to capture unique interactions but also increase the risk of over-fitting. Note that larger (both by number of features and observations) training data sets are more tolerable to deeper trees. __Recommendation__: Uniformly search across values ranging from 1-10 but be willing to increase the high value range for larger datasets.

* __min_child_weight__: Also, controls the complexity of each tree by requiring the minimum number of instances (measured by [hessian](https://stats.stackexchange.com/questions/317073/explanation-of-min-child-weight-in-xgboost-algorithm) within XGBoost) to be greater than a certain value for further partitioning to occur. Since we tend to use shorter trees this rarely has a large impact on performance but tuning it should not be overlooked. Typical values range from 5–15. Higher values help prevent a model from learning relationships which might be highly specific to the particular sample selected for a tree (overfitting), but smaller values can help with imbalanced target classes in classification problems. __Recommendation__: Uniformly search across values ranging from near zero-20 but be willing to increase the high value range for larger datasets.

__Stochastic hyperparameters__ - Stochastic gradient boosting is the act of randomly subsampling the training data set, whether that be by sampling rows or columns. Subsampling helps to reduce tree correlation and also helps reduce the chances of getting stuck in local minimas, plateaus, and other irregular terrain of the loss function so that we may find a near global optimum loss. There are four main ways we subsample while implementing XGBoost:

* __subsample__: Subsampling rows before creating each tree. This form of subsampling primarily helps to reduce tree correlation. Generally, aggressive subsampling of rows, such as selecting only 50% or less of the training data, has shown to be beneficial and typical values range between 0.5–0.8. __Recommendation__: Uniformly search across values ranging from 0.5-1.0.

* __colsample_bytree__: Subsampling of columns and the impact to performance largely depends on the nature of the data and if there is strong multicollinearity or a lot of noisy features. If there are fewer relevant predictors (more noisy data) higher values of column subsampling tends to perform better because it makes it more likely to select those features with the strongest signal. When there are many relevant predictors, a lower values of column subsampling tends to perform well. __Recommendation__: Uniformly search across values ranging from 0.5-1.0.

* __colsample_bylevel__ & __colsample_bynode__: These additional subsampling procedures act similar to `colsample_bytree`; however, they do additional subsampling of columns every time we achieve a new level of depth in a tree (`colsample_bylevel`) and/or every node split within a given tree depth level (`colsample_bynode`). These hyperparameters are less common but can be useful when dealing with extremely large data sets or with data sets that have very high multicollinearity. __Recommendation__: Uniformly search across values ranging from 0.5-1.0.

__Regularization hyperparameters__ - XGBoost provides multiple regularization parameters to help reduce model complexity and guard against overfitting. These include:

* __gamma__: A pseudo-regularization hyperparameter known as a Lagrangian multiplier that controls the complexity of a given tree. `gamma` specifies a minimum loss reduction required to make a further partition on a leaf node of the tree. When gamma is specified, XGBoost will grow the tree to the max depth but then prune the tree to find and remove splits that do not meet the specified gamma. gamma tends to be worth exploring as your trees in your GBM become deeper and when you see a significant difference between the train and validation/test error. The value of gamma typically ranges from \\(0 - ∞ \\)
  (0 means no constraint while large numbers mean a higher regularization). What quantifies as a large gamma value is dependent on the loss function but generally lower values between 1–20 will do if gamma is influential. __Recommendation__: Search across values ranging from 0-some large number on a log scale (i.e. 0, 1, 10, 100, 1000, etc.).

* __alpha__: Provides an \\( L_{2} \\) regularization to the loss function, which is similar to the [Ridge penalty](https://bradleyboehmke.github.io/HOML/regularized-regression.html#ridge) commonly used for regularized regression. Typically values range from \\(0 - ∞ \\) with larger values causing more conservative models. Setting both `alpha` and `lambda` to greater than 0 results in an [elastic net regularization](https://bradleyboehmke.github.io/HOML/regularized-regression.html#elastic). **Recommendation**: Search across values ranging from 0-some large number on a log scale (i.e. 0, 1, 10, 100, 1000, etc.).

* __lambda__: Provides an \\(L_{1} \\) regularization to the loss function, which is similar to the [Lasso penalty](https://bradleyboehmke.github.io/HOML/regularized-regression.html#lasso) commonly used for regularized regression. Typically values range from \\(0 - ∞ \\) with larger values causing more conservative models. Setting both `alpha` and `lambda` to greater than 0 results in an [elastic net regularization](https://bradleyboehmke.github.io/HOML/regularized-regression.html#elastic). **Recommendation**: Search across values ranging from 0-some large number on a log scale (i.e. 0, 1, 10, 100, 1000, etc.).



### Hyperopt

There are [several approaches](https://en.wikipedia.org/wiki/Hyperparameter_optimization) you can use for performing a hyperparameter grid search -- full cartesian grid search, random grid search, Bayesian optimization, etc. However, when performing a grid search over a large search space with many hyperparameters and value ranges, using a full cartesian search or a random unguided search will be very time consuming or lead to non-optimal results. To perform a grid search of this scale we prefer to use a ___Bayesian optimizer___, meaning it is not merely randomly searching or searching a grid, but intelligently learning which combinations of values work well as it goes, and focusing the search there.  

Multiple packages exist that provide Bayesian optimization grid search capabilities; however, [hyperopt](https://github.com/hyperopt/hyperopt) is one of the most popular packages and is also tightly integrated with Databricks and MLFlow. The benefits of using hyperopt include:

* Open source
* Bayesian optimizer – smart searches over hyperparameters (using a [Tree of Parzen Estimators](https://optunity.readthedocs.io/en/latest/user/solvers/TPE.html)), not grid or random search
* Integrates with Apache Spark for parallel hyperparameter search
* Integrates with MLflow for automatic tracking of the search results
* Included in the Databricks ML runtime
* Maximally flexible: can optimize literally any Python model with any hyperparameters

### Hyperparameter search space

Our first objective is to define our hyperparameter search space. Hyperopt provides a [variety of expressions](http://hyperopt.github.io/hyperopt/getting-started/search_spaces/) to create a range of values (i.e. `hp.loguniform`, `hp.uniform`). When establishing the range of values, choose bounds that are extreme and let hyperopt learn what values aren’t working well. For example, if a regularization parameter is typically between 1 and 10, try values from 0 to 100. The range should include the default value, certainly. At worst, it may spend time trying extreme values that do not work well at all, but it should learn and stop wasting trials on bad values. This may mean subsequently re-running the search with a narrowed range after an initial exploration to better explore reasonable values.

A best practice strategy for a hyperopt workflow is as follows:

1. Choose what hyperparameters are reasonable to optimize
2. Define broad ranges for each of the hyperparameters (including the default where applicable)
3. Run a small number of trials
4. Observe the results in an MLflow parallel coordinate plot and select the runs with lowest loss
5. Move the range towards those higher/lower values when the best runs’ hyperparameter values are pushed against one end of a range
6. Determine whether certain hyperparameter values cause fitting to take a long time (and avoid those values)
7. Re-run with more trials
8. Repeat until the best runs are comfortably within the given search bounds and none are taking excessive time

**NOTE**: when using `hp.loguniform` the values you specify for min and max should be expressed in `exp(x)`. For example, if you want the `learning_rate` to range from a min of 0.0001 and max of 1 on a log scale you would specify `hp.loguniform(-9, 0)` because `exp(-9) = 0.0001` and `exp(0) = 1`.

| Hyperparameter     | Suggested Hyperopt Expression | Value Ranges |
| ------------------ | --------------- | ------------------------------------ |
| `learning_rate`    | `hp.loguniform` | min = -9; max = 0 (because `exp(-9) = 0.0001` and `exp(0) = 1`) |
| `max_depth`        | `hp.uniform`    | min = 1; max = 100 (we wrap with `scope.int` since this must be an integer) |
| `min_child_weight` | `hp.loguniform` | min = -2; max = 3 (includes default value of 1) |
| `subsample`        | `hp.uniform`    | min = 0.5; max = 1 |
| `colsample_bytree` | `hp.uniform`    | min = 0.5; max = 1 |
| `gamma`            | `hp.loguniform` | min = -10; max = 10 (because `exp(-10) ~ 0` and `exp(10) > 10000`) |
| `alpha`            | `hp.loguniform` | min = -10; max = 10 (because `exp(-10) ~ 0` and `exp(10) > 10000`) |
| `lambda`           | `hp.loguniform` | min = -10; max = 10 (because `exp(-10) ~ 0` and `exp(10) > 10000`) |
  

In [0]:
search_space = {
  'learning_rate': hp.loguniform('learning_rate', -7, 0),
  'max_depth': scope.int(hp.uniform('max_depth', 1, 100)),
  'min_child_weight': hp.loguniform('min_child_weight', -2, 3),
  'subsample': hp.uniform('subsample', 0.5, 1),
  'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
  'gamma': hp.loguniform('gamma', -10, 10),
  'alpha': hp.loguniform('alpha', -10, 10),
  'lambda': hp.loguniform('lambda', -10, 10),
  'objective': 'binary:logistic',
  'eval_metric': 'auc',
  'seed': 123,
}

In [0]:
search_space

Out[11]: {'learning_rate': <hyperopt.pyll.base.Apply at 0x7fb1447cb550>,
 'max_depth': <hyperopt.pyll.base.Apply at 0x7fb1447cb580>,
 'min_child_weight': <hyperopt.pyll.base.Apply at 0x7fb1443230a0>,
 'subsample': <hyperopt.pyll.base.Apply at 0x7fb1443235b0>,
 'colsample_bytree': <hyperopt.pyll.base.Apply at 0x7fb144323670>,
 'gamma': <hyperopt.pyll.base.Apply at 0x7fb1443234c0>,
 'alpha': <hyperopt.pyll.base.Apply at 0x7fb1443236a0>,
 'lambda': <hyperopt.pyll.base.Apply at 0x7fb144323b80>,
 'objective': 'binary:logistic',
 'eval_metric': 'auc',
 'seed': 123}

### Defining the search process

When using hyperopt, we need to create an objective function that will be optimized by hyperopt's Bayesian optimizer. The function definition typically only requires one parameter (`params`), which hyperopt will use to pass a set of hyperparameter values. So given a set of hyperparameter values that hyperopt chooses, the function trains our given model and computes the loss for the model built with those hyperparameters. Consequently, we need our function to return a dictionary with at least two items:

* status: One of the keys from `hyperopt.STATUS_STRINGS` to signal successful vs. failed completion. Most common is `STATUS_OK`.
* loss: The loss score to be optimized. Note that hyperopt will always try to minimize the loss so if you choose a loss where the objective is to maximize (i.e. \\( R^2 \\), AUC) then you will need to negate this value.

A big benefit of using hyperopt is that it is automatically integrated with MLFlow. Consequently, we can leverage the many benefits of MLFlow within this process. For example, we can use:

* [`mlflow.xgboost.autolog`](https://www.mlflow.org/docs/latest/python_api/mlflow.xgboost.html#mlflow.xgboost.autolog) to automatically log the parameters that hyperopt chooses
* [`mlflow.start_run`](https://www.mlflow.org/docs/latest/python_api/mlflow.html#mlflow.start_run) to start a new MLFlow run to log to. This allows us to run different variations of hyperparameters runs and keep our logged values organized.
* [`mlflow.log_metric`](https://www.mlflow.org/docs/latest/python_api/mlflow.html#mlflow.log_metric) to record specific metrics of interest for each model run.

**Tip**: Since our learning rate hyperparameter can very from very small to quite large, it is always recommended to crank up the number of boosted trees (`num_boost_round`) and use `early stopping_rounds` to end training once the loss has stopped improving for a given model run.  Also, some hyperparameters have a large impact on runtime. A very small learning rate and large max tree depth can cause it to fit models that are large and expensive to train. Worse, sometimes models take a long time to train because they are overfitting the data! hyperopt does not try to learn about runtime of trials or factor that into its choice of hyperparameters. Consequently, it can be beneficial to time the run time for each model instance so you can make a tradeoff between performance and compute time.


In [0]:
def train_model(params):
  # With MLflow autologging, hyperparameters and the trained model are automatically logged to MLflow.
  mlflow.xgboost.autolog(silent=True)
  
  # However, we can log additional information by using an MLFlow tracking context manager 
  with mlflow.start_run(nested=True):

    # Train model and record run time
    start_time = time.time()
    booster = xgb.train(params=params, dtrain=train, num_boost_round=5000, evals=[(test, "test")], early_stopping_rounds=50, verbose_eval=False)
    run_time = time.time() - start_time
    mlflow.log_metric('runtime', run_time)
    
    # Record AUC as primary loss for Hyperopt to minimize
    predictions_test = booster.predict(test)
    auc_score = roc_auc_score(y_test, predictions_test)
    
    # Set the loss to -1*auc_score so fmin maximizes the auc_score
    return {'status': STATUS_OK, 'loss': -auc_score, 'booster': booster.attributes()}

### Executing the search

To execute the search we use `fmin` and supply it our model training (objective) function along with the hyperparameter search space. `fmin` can use different algorithms to search across the hyperparameter search space (i.e. random, Bayesian); however, we suggest using the Tree of Parsen Estimators (`tpe.suggest`) which will perform a smart Bayesian optimization grid search.  

Also, as previously mentioned, its is smart to run an initial small number of trials to find a range of hyperparameter values that appear to perform well and then refine the search space. The following example runs 25 trials for the initial search (note that 'small' will be relative to compute time and size of the search space).

**Note**: Hyperopt can parallelize its trials across a Spark cluster, which is a great feature. Building and evaluating a model for each set of hyperparameters is inherently parallelizable, as each trial is independent of the others. Using Spark to execute trials is simply a matter of using “SparkTrials” in Hyperopt. This is a great idea in environments like Databricks where a Spark cluster is readily available. However, considering this notebook is using a single node cluster there are no additional benefits in trying to parallelize.

In [0]:
#spark_trials = SparkTrials(parallelism=4)

In [0]:
with mlflow.start_run(run_name='initial_search'):
  best_params = fmin(
    fn=train_model, 
    space=search_space, 
    algo=tpe.suggest, 
    max_evals=25,
    rstate=np.random.RandomState(123),
    #trials=spark_trials
  )

0%| | 0/25 [00:00<?, ?trial/s, best loss=?] 4%|▍ | 1/25 [00:38<15:30, 38.78s/trial, best loss: -0.8832969520575632] 8%|▊ | 2/25 [00:43<11:00, 28.70s/trial, best loss: -0.8832969520575632] 12%|█▏ | 3/25 [00:48<07:51, 21.45s/trial, best loss: -0.8832969520575632] 16%|█▌ | 4/25 [00:57<06:14, 17.83s/trial, best loss: -0.8938380105862379] 20%|██ | 5/25 [01:06<05:00, 15.04s/trial, best loss: -0.9133840293332953] 24%|██▍ | 6/25 [01:12<03:53, 12.27s/trial, best loss: -0.9145444255800791] 28%|██▊ | 7/25 [02:15<08:14, 27.46s/trial, best loss: -0.9145444255800791] 32%|███▏ | 8/25 [03:41<12:48, 45.21s/trial, best loss: -0.9145444255800791] 36%|███▌ | 9/25 [03:44<08:41, 32.61s/trial, best loss: -0.9145444255800791] 40%|████ | 10/25 [04:33<09:20, 37.37s/trial, best loss: -0.9145444255800791] 44%|████▍ | 11/25 [04:39<06:33, 28.12s/trial, best loss: -0.9145444255800791] 48%|████▊ | 12/25 [04:43<04:29, 20.72s/trial, best loss: -0.9145444255800791] 52%|█████▏ | 13/25 [04:49<03:14, 16.19s/trial, best loss: -0.9145444255800791] 56%|█████▌ | 14/25 [04:51<02:14, 12.21s/trial, best loss: -0.9145444255800791] 60%|██████ | 15/25 [05:25<03:06, 18.68s/trial, best loss: -0.9145444255800791] 64%|██████▍ | 16/25 [05:32<02:15, 15.07s/trial, best loss: -0.9145444255800791] 68%|██████▊ | 17/25 [05:37<01:37, 12.23s/trial, best loss: -0.9145444255800791] 72%|███████▏ | 18/25 [05:41<01:08, 9.74s/trial, best loss: -0.9145444255800791] 76%|███████▌ | 19/25 [05:46<00:49, 8.33s/trial, best loss: -0.9145444255800791] 80%|████████ | 20/25 [05:49<00:33, 6.72s/trial, best loss: -0.9145444255800791] 84%|████████▍ | 21/25 [05:52<00:22, 5.56s/trial, best loss: -0.9145444255800791] 88%|████████▊ | 22/25 [05:57<00:16, 5.35s/trial, best loss: -0.9145444255800791] 92%|█████████▏| 23/25 [06:00<00:09, 4.64s/trial, best loss: -0.9145444255800791] 96%|█████████▌| 24/25 [06:04<00:04, 4.47s/trial, best loss: -0.9145444255800791]100%|██████████| 25/25 [06:09<00:00, 4.71s/trial, best loss: -0.9145444255800791]100%|██████████| 25/25 [06:09<00:00, 14.80s/trial, best loss: -0.9145444255800791]

In [0]:
best_params

Out[15]: {'alpha': 1.2573697498285759,
 'colsample_bytree': 0.6246623099667723,
 'gamma': 0.4299177395431556,
 'lambda': 0.6655776343087407,
 'learning_rate': 0.10108159135348746,
 'max_depth': 8.571533913539605,
 'min_child_weight': 1.3053934392357864,
 'subsample': 0.6654338738457878}

### Assess results

As shown above, `best_params` will contain the hyperparameter values that results in the highest AUC. However, we can also click on the "experiment in MLflow" link which will take us to MLFlow experiment board where additional details can be found. 

For example, we can compare all the model runs and hyperparameter values with a parallel coordinates plot to assess which hyperparameter values tend to be most related high AUC scores. In the below plot we can see that those models with higher AUCs tend to have:

* higher `max_depth` values
* lower `min_child_weight`, `learning_rate`, `lambda`, `gamma`, and `alpha` values
* and it does not appear that `subsample` and `colsample_bytree` have much effect

![hyperparameter_results](https://user-images.githubusercontent.com/6753598/132388405-fb243df5-4a6d-43c8-a33a-f8e82ffc1f3a.png)

Also, we can look at a scatter plot of AUC vs. runtime and we see that some models tend to perform quite well but with substantially less runtime demands than other high performing models. The objective is to use the findings from these plots to identify refined values for our search space and then perform a second iteraction of executing a search to try and find a more optimal model. However, for brevity we will leave this as an exercise for our reader.

![runtime_performance](https://user-images.githubusercontent.com/6753598/132387421-875873d1-f415-46bb-ac30-7203f7e93aef.png)

### Alternative early stopping procedures

With GBMs (and a few other resource demanding algorithms such as deep learning), compute time can increase significantly as the dimensions of the data and/or hyperparameter grid search expand. Hyperopt provides a few alternative methods for early stopping procedures. In addition of using `max_evals` for limiting how many models to run, we can also stop our hyperparameter search using `timeout` (stop based on time) and `loss_threshold` (stop once we've met a certain loss value), or a combination of any of the above.

In [0]:
with mlflow.start_run(run_name='xgb_timeout'):
  best_params = fmin(
    fn=train_model, 
    space=search_space, 
    algo=tpe.suggest, 
    timeout=60*5, # stop the grid search after 5 * 60 seconds == 5 minutes
    #trials=spark_trials, 
    rstate=np.random.RandomState(123)
  )

0%| | 0/9223372036854775807 [00:00<?, ?trial/s, best loss=?] 0%| | 1/9223372036854775807 [00:37<96753193278784580:16:00, 37.76s/trial, best loss: -0.8832969520575632] 0%| | 2/9223372036854775807 [00:42<71646566171643567:13:04, 27.96s/trial, best loss: -0.8832969520575632] 0%| | 3/9223372036854775807 [00:47<53548798805602522:27:12, 20.90s/trial, best loss: -0.8832969520575632] 0%| | 4/9223372036854775807 [00:56<44632264026244683:05:36, 17.42s/trial, best loss: -0.8938380105862379] 0%| | 5/9223372036854775807 [01:04<37677797860577562:10:08, 14.71s/trial, best loss: -0.9133840293332953] 0%| | 6/9223372036854775807 [01:10<30814597250650562:33:36, 12.03s/trial, best loss: -0.9145444255800791] 0%| | 7/9223372036854775807 [02:11<68479127586968243:46:08, 26.73s/trial, best loss: -0.9145444255800791] 0%| | 8/9223372036854775807 [03:35<112506087068127796:20:16, 43.91s/trial, best loss: -0.9145444255800791] 0%| | 9/9223372036854775807 [03:38<81066278264448923:52:32, 31.64s/trial, best loss: -0.9145444255800791]  0%| | 10/9223372036854775807 [04:27<93836703760242442:14:24, 36.63s/trial, best loss: -0.9145444255800791] 0%| | 11/9223372036854775807 [04:33<70792878146952788:11:44, 27.63s/trial, best loss: -0.9145444255800791] 0%| | 12/9223372036854775807 [04:37<52201162241687397:15:44, 20.37s/trial, best loss: -0.9145444255800791] 0%| | 13/9223372036854775807 [04:42<40973302831781073:21:04, 15.99s/trial, best loss: -0.9145444255800791] 0%| | 14/9223372036854775807 [04:45<30813075729252497:38:08, 12.03s/trial, best loss: -0.9145444255800791] 0%| | 15/9223372036854775807 [05:19<47414659078308813:56:16, 18.51s/trial, best loss: -0.9145444255800791] 0%| | 15/9223372036854775807 [05:19<54535739720798126:04:48, 21.29s/trial, best loss: -0.9145444255800791]

In [0]:
with mlflow.start_run(run_name='xgb_loss_threshold'):
  best_params = fmin(
    fn=train_model, 
    space=search_space, 
    algo=tpe.suggest, 
    loss_threshold=-0.92, # stop the grid search once we've reached an AUC of 0.92 or higher
    timeout=60*10,        # stop after 5 minutes regardless if we reach an AUC of 0.92
    #trials=spark_trials, 
    rstate=np.random.RandomState(123)
  )

0%| | 0/9223372036854775807 [00:00<?, ?trial/s, best loss=?] 0%| | 1/9223372036854775807 [00:37<96240238119078242:59:12, 37.56s/trial, best loss: -0.8832969520575632] 0%| | 2/9223372036854775807 [00:42<71309962414923967:08:48, 27.83s/trial, best loss: -0.8832969520575632] 0%| | 3/9223372036854775807 [00:47<53319491220932926:34:40, 20.81s/trial, best loss: -0.8832969520575632] 0%| | 4/9223372036854775807 [00:55<44036831211799092:20:16, 17.19s/trial, best loss: -0.8938380105862379] 0%| | 5/9223372036854775807 [01:04<37199109611504030:09:04, 14.52s/trial, best loss: -0.9133840293332953] 0%| | 6/9223372036854775807 [01:09<30535095605831229:26:24, 11.92s/trial, best loss: -0.9145444255800791] 0%| | 7/9223372036854775807 [02:09<67236595680661285:32:48, 26.24s/trial, best loss: -0.9145444255800791] 0%| | 8/9223372036854775807 [03:33<111542934493929472:00:00, 43.54s/trial, best loss: -0.9145444255800791] 0%| | 9/9223372036854775807 [03:36<80295007675521642:57:04, 31.34s/trial, best loss: -0.9145444255800791]  0%| | 10/9223372036854775807 [04:28<95865764259167964:43:44, 37.42s/trial, best loss: -0.9145444255800791] 0%| | 11/9223372036854775807 [04:34<71913581953903019:48:16, 28.07s/trial, best loss: -0.9145444255800791] 0%| | 12/9223372036854775807 [04:37<52857712435009654:19:44, 20.63s/trial, best loss: -0.9145444255800791] 0%| | 13/9223372036854775807 [04:43<41478364501039222:19:44, 16.19s/trial, best loss: -0.9145444255800791] 0%| | 14/9223372036854775807 [04:46<31169638706418888:14:56, 12.17s/trial, best loss: -0.9145444255800791] 0%| | 15/9223372036854775807 [05:19<47303719084782969:44:32, 18.46s/trial, best loss: -0.9145444255800791] 0%| | 16/9223372036854775807 [05:25<37794268894836895:17:20, 14.75s/trial, best loss: -0.9145444255800791] 0%| | 17/9223372036854775807 [05:30<30731327040806766:21:52, 11.99s/trial, best loss: -0.9145444255800791] 0%| | 18/9223372036854775807 [05:34<24398828282182078:00:32, 9.52s/trial, best loss: -0.9145444255800791] 0%| | 19/9223372036854775807 [05:39<20717729122644409:27:28, 8.09s/trial, best loss: -0.9145444255800791] 0%| | 20/9223372036854775807 [05:42<16892792095345404:35:12, 6.59s/trial, best loss: -0.9145444255800791] 0%| | 21/9223372036854775807 [05:45<13952929479930154:05:52, 5.45s/trial, best loss: -0.9145444255800791] 0%| | 22/9223372036854775807 [05:50<13953931174390340:16:00, 5.45s/trial, best loss: -0.9145444255800791] 0%| | 23/9223372036854775807 [05:54<12383790334478661:24:16, 4.83s/trial, best loss: -0.9145444255800791] 0%| | 24/9223372036854775807 [05:58<11887509991584956:52:16, 4.64s/trial, best loss: -0.9145444255800791] 0%| | 25/9223372036854775807 [06:03<12113493855533697:42:24, 4.73s/trial, best loss: -0.9145444255800791] 0%| | 26/9223372036854775807 [06:18<20500164249538546:20:48, 8.00s/trial, best loss: -0.9145444255800791] 0%| | 27/9223372036854775807 [06:28<21674458173540920:53:20, 8.46s/trial, best loss: -0.9145444255800791] 0%| | 28/9223372036854775807 [06:31<17306897863352249:27:28, 6.76s/trial, best loss: -0.9145444255800791] 0%| | 29/9223372036854775807 [07:27<55642367759014170:10:08, 21.72s/trial, best loss: -0.9145444255800791] 0%| | 30/9223372036854775807 [07:38<47072223898454735:04:32, 18.37s/trial, best loss: -0.9145444255800791] 0%| | 31/9223372036854775807 [07:50<42066076570692912:55:28, 16.42s/trial, best loss: -0.9145444255800791] 0%| | 32/9223372036854775807 [09:39<113476772799893704:14:56, 44.29s/trial, best loss: -0.9210478758566245] 0%| | 32/9223372036854775807 [09:39<46409137786372460:05:20, 18.11s/trial, best loss: -0.9210478758566245]


## Summary

Tuning an XGBoost algorithm is no small feat. This tutorial outlined the primary hyperparameters that tend to impact model performance along with recommended values to explore for each hyperparameter. In addition, this tutorial illustrated how to use Hyperopt for an intelligent, Bayesian optimization approach to explore the search space along with MLFlow to log and organize the hyperparameter exploration within Databricks.

However, this tutorial should only serve as an introduction to these tools and processes. To go further and learn more please explore the following additional resources.


## Additional resources

* XGBoost
   - [Getting started with XGBoost on Databricks](https://docs.databricks.com/applications/machine-learning/train-model/xgboost.html) (Databricks docs)
   - [Additional notes on XGBoost hyperparameter tuning](https://xgboost.readthedocs.io/en/latest/tutorials/param_tuning.html) (package docs)
   - [A Gentle Introduction to XGBoost for Applied Machine Learning](https://machinelearningmastery.com/gentle-introduction-xgboost-applied-machine-learning/) (blog)
   
* Hyperopt
   - [Hyperopt best practices documentation from Databricks](https://docs.databricks.com/applications/machine-learning/automl-hyperparam-tuning/hyperopt-best-practices.html?_gl=1*wlclzc*_gcl_aw*R0NMLjE2Mjg3OTM0NTQuQ2p3S0NBandqZE9JQmhBX0Vpd0FIejh4bTF0d0hKSFdvYzgzUDBHTWd5Z3duMFZMb1dueUpDYVU0aDZSOHdDVE91UU05VWZ4QTdEaUxSb0NPN1FRQXZEX0J3RQ..&_ga=2.244655938.636196059.1630927467-1465050559.1628793454&_gac=1.16404164.1628793456.CjwKCAjwjdOIBhA_EiwAHz8xm1twHJHWoc83P0GMgygwn0VLoWnyJCaU4h6R8wCTOuQM9UfxA7DiLRoCO7QQAvD_BwE) (Databricks docs)
   - [How (Not) to Tune Your Model With Hyperopt](https://databricks.com/blog/2021/04/15/how-not-to-tune-your-model-with-hyperopt.html) (blog)
   - [Scaling Hyperopt to Tune Machine Learning Models in Python](https://databricks.com/blog/2019/10/29/scaling-hyperopt-to-tune-machine-learning-models-in-python.html) (blog)
* MLFlow
   - [Python MLFlow API documentation](https://www.mlflow.org/docs/latest/python_api/index.html) (package docs)
   - [MLFlow guide](https://docs.databricks.com/applications/mlflow/index.html) (Databricks docs)
   - [Best Practices for Hyperparameter Tuning with MLflow](https://databricks.com/session/best-practices-for-hyperparameter-tuning-with-mlflow) (talk)